In [1]:
from utils.tensorrt import InferenceModel

In [2]:
from pycuda import autoinit

In [3]:
model = InferenceModel(
    "models/retinanet_540p.onnx",
    "models/retinanet_540p.engine",
    force_rebuild=True,
    fp16_mode=True,
)

Building TensorRT engine from models/retinanet_540p.onnx...
Initializing memory buffers...
